In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

from vqf import get_classical_energy, get_pauli_str, get_cost_hamiltonian

from qiskit.utils import algorithm_globals
from qiskit.algorithms.optimizers import COBYLA
from qiskit import Aer, transpile
from qiskit.algorithms import QAOA

Consider the factoring of the biprime

$$
m = p * q
$$

where, 

$$
m = \sum_{k=0}^{n_m -1} 2^k m_k \;\;\;, m_k \in \{0,1 \}
$$ 

$$
p = \sum_{k=0}^{n_p -1} 2^k p_k \;\;\;, p_k \in \{0,1 \}
$$ 

$$
q = \sum_{k=0}^{n_q -1} 2^k q_k \;\;\;, q_k \in \{0,1 \}
$$ 

We assume that 

$$
p \geq q
$$

$$
n_p = n_m
$$

$$
n_q = \left \lceil \frac{n_m}{2} \right \rceil
$$

Then,

$$
n_c = n_p + n_q - 1
$$

In [2]:
p = 43
q = 37
m = p * q

The set of clauses for the factoring problem 
$$
C_i = \sum _{j = 0} ^ i q_j p_{i-j} + \sum _{j = 0} ^ i z_{j,i} - m_i - \sum _{j = 0} ^ i 2^j z_{i, i+j} \;\;\;, 0 \leq i < n_c
$$

Hence, the classical energy is defined as

$$
E = \sum_{i=0} ^ {n_c} C_i^2
$$

In [4]:
energy = get_classical_energy(m)
energy

p_0**2*q_0**2 + p_0**2*q_1**2 + p_0**2*q_2**2 + p_0**2*q_3**2 + p_0**2*q_4**2 + p_0**2*q_5**2 + 2*p_0*p_1*q_0*q_1 + 2*p_0*p_1*q_1*q_2 + 2*p_0*p_1*q_2*q_3 + 2*p_0*p_1*q_3*q_4 + 2*p_0*p_1*q_4*q_5 + 2*p_0*p_2*q_0*q_2 + 2*p_0*p_2*q_1*q_3 + 2*p_0*p_2*q_2*q_4 + 2*p_0*p_2*q_3*q_5 + 2*p_0*p_3*q_0*q_3 + 2*p_0*p_3*q_1*q_4 + 2*p_0*p_3*q_2*q_5 + 2*p_0*p_4*q_0*q_4 + 2*p_0*p_4*q_1*q_5 + 2*p_0*p_5*q_0*q_5 - 2*p_0*q_0 - 1024*p_0*q_1*z_1_10 - 4*p_0*q_1*z_1_2 - 8*p_0*q_1*z_1_3 - 16*p_0*q_1*z_1_4 - 32*p_0*q_1*z_1_5 - 64*p_0*q_1*z_1_6 - 128*p_0*q_1*z_1_7 - 256*p_0*q_1*z_1_8 - 512*p_0*q_1*z_1_9 - 2*p_0*q_1 + 2*p_0*q_2*z_1_2 - 512*p_0*q_2*z_2_10 - 4*p_0*q_2*z_2_3 - 8*p_0*q_2*z_2_4 - 16*p_0*q_2*z_2_5 - 32*p_0*q_2*z_2_6 - 64*p_0*q_2*z_2_7 - 128*p_0*q_2*z_2_8 - 256*p_0*q_2*z_2_9 - 2*p_0*q_2 + 2*p_0*q_3*z_1_3 + 2*p_0*q_3*z_2_3 - 256*p_0*q_3*z_3_10 - 4*p_0*q_3*z_3_4 - 8*p_0*q_3*z_3_5 - 16*p_0*q_3*z_3_6 - 32*p_0*q_3*z_3_7 - 64*p_0*q_3*z_3_8 - 128*p_0*q_3*z_3_9 + 2*p_0*q_4*z_1_4 + 2*p_0*q_4*z_2_4 + 2*p_0*q_4*z_3_4

We apply the following classical preprocessing rules

$$
x y - 1 = 0 \Rightarrow x = y = 1,
$$

$$
x + y − 1 = 0 \Rightarrow xy = 0,
$$

$$
a − bx = 0 \Rightarrow x = 1,
$$

$$
\sum_i x_i = 0 \Rightarrow x_i = 0,
$$

$$
\sum_{i=1}^a x_i − a = 0 \Rightarrow x_i = 1.
$$

to the classical energy function to get the simplified energy function

$$
E^{'} = \sum_{i=0} ^ {n_c} C_i^{'2}
$$

In [5]:
energy = get_classical_energy(m, apply_rules=True)
energy

2*p_0*p_1*q_0*q_1 + 2*p_0*p_1*q_1*q_2 + 2*p_0*p_1*q_2*q_3 + 2*p_0*p_1*q_3*q_4 + 2*p_0*p_1*q_4*q_5 + 2*p_0*p_2*q_0*q_2 + 2*p_0*p_2*q_1*q_3 + 2*p_0*p_2*q_2*q_4 + 2*p_0*p_2*q_3*q_5 + 2*p_0*p_3*q_0*q_3 + 2*p_0*p_3*q_1*q_4 + 2*p_0*p_3*q_2*q_5 + 2*p_0*p_4*q_0*q_4 + 2*p_0*p_4*q_1*q_5 + 2*p_0*p_5*q_0*q_5 - p_0*q_0 - 1024*p_0*q_1*z_1_10 - 4*p_0*q_1*z_1_2 - 8*p_0*q_1*z_1_3 - 16*p_0*q_1*z_1_4 - 32*p_0*q_1*z_1_5 - 64*p_0*q_1*z_1_6 - 128*p_0*q_1*z_1_7 - 256*p_0*q_1*z_1_8 - 512*p_0*q_1*z_1_9 - p_0*q_1 + 2*p_0*q_2*z_1_2 - 512*p_0*q_2*z_2_10 - 4*p_0*q_2*z_2_3 - 8*p_0*q_2*z_2_4 - 16*p_0*q_2*z_2_5 - 32*p_0*q_2*z_2_6 - 64*p_0*q_2*z_2_7 - 128*p_0*q_2*z_2_8 - 256*p_0*q_2*z_2_9 - p_0*q_2 + 2*p_0*q_3*z_1_3 + 2*p_0*q_3*z_2_3 - 256*p_0*q_3*z_3_10 - 4*p_0*q_3*z_3_4 - 8*p_0*q_3*z_3_5 - 16*p_0*q_3*z_3_6 - 32*p_0*q_3*z_3_7 - 64*p_0*q_3*z_3_8 - 128*p_0*q_3*z_3_9 + p_0*q_3 + 2*p_0*q_4*z_1_4 + 2*p_0*q_4*z_2_4 + 2*p_0*q_4*z_3_4 - 128*p_0*q_4*z_4_10 - 4*p_0*q_4*z_4_5 - 8*p_0*q_4*z_4_6 - 16*p_0*q_4*z_4_7 - 32*p_0*q_4*z_

We make the ising substitution for each term of the classical energy $E^{'}$

$$
b_k \rightarrow \frac{1}{2} \left( 1 - \sigma_{b, k}^z \right)
$$

In [6]:
bits = energy.free_symbols
n_bits = len(bits)
print(f"The total number of classical bits: {n_bits}")

The total number of classical bits: 62


In [7]:
for a in range(len(energy.args)):
    term = energy.args[a]
    print(term)
    ising = get_pauli_str(term, bits)
    print(str(ising) + "\n")

7
None

3*z_3_4
1.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
- 1.5 * IIIIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

3*z_8_9
1.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
- 1.5 * IIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

5*z_6_7
2.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
- 2.5 * IIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

5*z_7_8
2.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
- 2.5 * IIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

7*z_1_2
3.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
- 3.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIII

7*z_4_5
3.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
- 3.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZII

7*z_9_10
3.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
- 3.5 * IIIIIIIIIIII

By summing those terms we get the factoring cost hamiltonian 

$$
H = \sum_{i=0} ^ {n_c} C_i^{'2}
$$

In [8]:
H = get_cost_hamiltonian(m)
print(str(H))

463412.75 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
+ 12.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIII
- 6.75 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIIIII
+ 1.0 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIZIIIIIIIIIIII
+ 43.25 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIII
+ 1.25 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIZIIIIIIIIIIII
+ 1.125 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIZIIIIIIIIIIIIIIIIIII
- 0.25 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIZIIIIIIZIIIIIIIIIIII
- 7.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
+ 1.125 * IIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIII
+ 0.625 * IIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIIIII
- 0.125 * IIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIZIIIIIIZIIIIIIIIIIII
+ 1.25 * IIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIII
- 0.25 * IIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIZIIIIIIIIIIII

In [9]:
algorithm_globals.random_seed = 10598
optimizer = COBYLA()
instance = Aer.get_backend("aer_simulator")
qaoa = QAOA(optimizer, quantum_instance=instance)
# result = qaoa.compute_minimum_eigenvalue(H)

### Before transpilation

In [10]:
params = np.ones(2)  # a typical QAOA ansatz has two parameters, we set them to be ones.
qc = qaoa.construct_circuit(params, H)
qc = qc[0]
# qc = qc.decompose().decompose()#.decompose()
# qc.draw('mpl')

In [11]:
n_qubits = qc.num_qubits
depth = qc.depth()
gates = qc.count_ops()
# n_cnots = gates['cx']
# n_u3 = gates['u3']
# n_gates = n_cnots + n_u3

print(
    f"The circuit parameters for factoring the biprime {m} using QAOA (no transpilation) \n"
)
print(f"The depth of the circuit: {depth}")
print(f"The total number of qubits: {n_qubits}")
print(f"The total number of gates: {str(gates)}")
# print(f"The total number of U gates: {n_u3}")
# print(f"The total number of CX gates: {n_cnots}")

The circuit parameters for factoring the biprime 1591 using QAOA (no transpilation) 

The depth of the circuit: 3
The total number of qubits: 62
The total number of gates: OrderedDict([('h', 62), ('PauliEvolution', 2)])


In [12]:
for level in range(4):
    t_circ = transpile(qc, basis_gates=["cx", "u3"], optimization_level=level)
    n_qubits = t_circ.num_qubits
    depth = t_circ.depth()
    gates = t_circ.count_ops()
    n_cnots = gates["cx"]
    n_u3 = gates["u3"]
    n_gates = n_cnots + n_u3

    print(
        f"The circuit parameters for factoring the biprime {m} using QAOA  (transpilation level: {level}) \n"
    )
    print(f"The depth of the circuit: {depth}")
    print(f"The total number of qubits: {n_qubits}")
    print(f"The total number of gates: {n_gates}")
    print(f"The total number of U gates: {n_u3}")
    print(f"The total number of CX gates: {n_cnots}")
    print("--------------------------------------------")

The circuit parameters for factoring the biprime 1591 using QAOA  (transpilation level: 0) 

The depth of the circuit: 4182
The total number of qubits: 62
The total number of gates: 8439
The total number of U gates: 2219
The total number of CX gates: 6220
--------------------------------------------
The circuit parameters for factoring the biprime 1591 using QAOA  (transpilation level: 1) 

The depth of the circuit: 3651
The total number of qubits: 62
The total number of gates: 7839
The total number of U gates: 2157
The total number of CX gates: 5682
--------------------------------------------
The circuit parameters for factoring the biprime 1591 using QAOA  (transpilation level: 2) 

The depth of the circuit: 3649
The total number of qubits: 62
The total number of gates: 7837
The total number of U gates: 2157
The total number of CX gates: 5680
--------------------------------------------
The circuit parameters for factoring the biprime 1591 using QAOA  (transpilation level: 3) 

The 